<h1> Instalando e importando as bibliotecas 


In [1]:
#importa libraries necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import warnings
warnings.filterwarnings('ignore')

<h1> Carregando os dados da planilha



In [2]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download dos dados para esta máquina virtual
downloaded = drive.CreateFile({'id': '1ZOspzswkB75WFfSg23GrCMdqIOf0EXOH'})
downloaded.GetContentFile('data.csv')

In [3]:
#Carregando os dados baixados no dataframe
df = pd.read_csv('data.csv')

In [4]:
#olhando as colunas do df
df.columns

Index(['anomes', 'num_cpf_hash', 'vlr_credito', 'vlr_saldo', 'num_atend_atrs',
       'vlr_score', 'num_produtos', 'num_atend', 'qtd_oper', 'qtd_reclm',
       'qtd_restr', 'vlr_renda', 'cod_rating', 'ind_atrito', 'ind_engaj',
       'ind_novo_cli'],
      dtype='object')

#Criação de um data frame com um cliente por linha que em algum momento foram engajados com o banco, resumindo a sua trajetória durante os 12 mêses fornecidos pelo banco

<h2>1) Precisamos pegar toda a jornada dos clientes que em algum momento ja foram engajados com o banco

<p>1º) Criamos um df que contenha apenas um cpf por linha de pessoas que possuem o indice engajado como 1, independente do anomes

In [5]:
#df com pessoas que em algum momentos foram engajadas 
df_engajado = df[(df.ind_engaj == 1)]. drop_duplicates("num_cpf_hash")

<p>2º) Criamos um df que pegará todos os anomes daquelas pessoas que possuem indice engajado como 1, independente se naquele anomes o indice engajado estava 1 ou 0.

In [6]:
#Retiramos todos os valores Na das colunas "cod_rating" e "num_produtos"
df = df.dropna(subset=['cod_rating', 'num_produtos'])

In [7]:
#Somamos todos os valores NA de cada coluna para verificar se existe tais lavores nas colunas "cod_rating" e "num_produtos"
df.isna().sum()

anomes                  0
num_cpf_hash            0
vlr_credito        121692
vlr_saldo               0
num_atend_atrs    5445969
vlr_score          258413
num_produtos            0
num_atend         5428632
qtd_oper           121692
qtd_reclm         5451166
qtd_restr         3214699
vlr_renda         5046419
cod_rating              0
ind_atrito        5449051
ind_engaj         3294365
ind_novo_cli      5451994
dtype: int64

In [8]:
#-------------------------------------------------------------
##criação de um df, que não é um ponteiro para outro df, que possui os dados de todos os anomes dos clientes ja foram engajados em algum momento, eliminando as colunas "vlr_renda", "ind_novo_cli" e "anomes". Essas colunas foram removidas pois acreditamos que não agregam para nosso resultado de indice de engajamento. Também já substituimos os valores Nan por 0. 
clientes_engajados_jornada = df[df["num_cpf_hash"].isin(df_engajado.num_cpf_hash.values)].drop(["vlr_renda", "ind_novo_cli", "anomes"], axis = 1).copy()

In [9]:
#deletamos algumas variáveis para liberar espaço na memória ram 
del df 
del df_engajado

In [10]:
# verificamos se todas as colunas que desejamos está no df
clientes_engajados_jornada.columns

Index(['num_cpf_hash', 'vlr_credito', 'vlr_saldo', 'num_atend_atrs',
       'vlr_score', 'num_produtos', 'num_atend', 'qtd_oper', 'qtd_reclm',
       'qtd_restr', 'cod_rating', 'ind_atrito', 'ind_engaj'],
      dtype='object')

In [11]:
#Somamos todos os valores NA da coluna para verificar a existencia desses valores 
clientes_engajados_jornada['num_produtos'].isna().sum()

0

In [12]:
#Somamos todos os valores NA da coluna para verificar a existencia desses valores 
clientes_engajados_jornada['cod_rating'].isna().sum()

0

In [13]:
#Checagem de quantas linhas temos no df que acabamos de criar 
len(clientes_engajados_jornada)

3258559

<h2>2) Precisamos filtrar essas informações aplicando algumas "feature engineering" e resumir os clientes em uma única linha

<p>1º) Criar um label encoder para a coluna que temos string como valor

In [14]:
#Checagem de quais valores temos na coluna "cod_rating"
clientes_engajados_jornada["cod_rating"].unique()

array(['A', 'C', 'B', 'E', 'D', 'F', 'H', 'HH', 'G', 'AA'], dtype=object)

In [15]:
# aplica label encode em cod_rating já que o valor da coluna não é numérico,mas sim texto
clientes_engajados_jornada["cod_rating"] = clientes_engajados_jornada["cod_rating"].astype('category')

#poem os novos valores na coluna
clientes_engajados_jornada["cod_rating"] = clientes_engajados_jornada["cod_rating"].cat.codes
clientes_engajados_jornada["cod_rating"].unique()

array([0, 3, 2, 5, 4, 6, 8, 9, 7, 1], dtype=int8)

<p>2º) Ajuste de incongruência nos dados de saldo e número de produtos

In [16]:
#  filtramos clientes que tem saldo mas não tem produtos já que ter saldo é um produto. Substituimos 0 por 1(não realizamos a mudança pois ainda não confirmamos com o cliente se nossa hipótese está certa)

# clientes_engajados_jornada.isin(clientes_engajados_jornada[(clientes_engajados_jornada["num_produtos"] ==0.0) & (clientes_engajados_jornada["vlr_saldo"] !=0)])["num_produtos"].replace(to_replace=0.0,value=1,inplace=True)
#clientes_engajados_jornada[(clientes_engajados_jornada["num_produtos"] ==0.0) & (clientes_engajados_jornada["vlr_saldo"] !=0)]

<p>3º) Realização de soma, mediana, média e valor máximo para cada histórico dos clientes ao longo dos anomes disponíveis de cada

<p>Realizamos os seguintes agrupamentos para as seguintes colunas:
<p>-------------------------
<p>Mediana:
<p>qtd_oper
<p>qtd_reclm
<p>num_atend_atrs
<p>num_atend
<p>qtd_restr
<p>vlr_credito
<p>vlr_saldo
<p>num_produtos
<p>ind_engaj
<p>cod_rating
<p>-------------------------
<p>Máximo:
<p>ind_atrito
<p>-------------------------
<h5>Média:
<p>vlr_score



In [17]:
#Criação de uma função para realizar os agrupamentos que precisamos 
def agrupar_dados(this_df):
    this_df["qtd_oper"] = this_df.qtd_oper.median()
    this_df["qtd_reclm"] = this_df.qtd_reclm.median()
    this_df["num_atend_atrs"] = this_df.num_atend_atrs.median()
    this_df["num_atend"] = this_df.num_atend.median()
    this_df["restr"] = this_df.qtd_restr.median()

     #Pegando a mediana dos valores
    this_df["vlr_credito"] = this_df.vlr_credito.median()
    this_df["vlr_saldo"] = this_df.vlr_saldo.median()
    this_df["num_produtos"] = this_df.num_produtos.median()
    this_df["cod_rating"] = this_df.cod_rating.median()
    this_df["ind_engaj"] = this_df.ind_engaj.median()

    #Pegando a média dos valores
    this_df["vlr_score"] = this_df.vlr_score.mean()

    #Pegando o máximo dos valores
    this_df["ind_atrito"] = this_df.ind_atrito.max()
   
   #Retornamos o df modificado
    return this_df

In [18]:
# Aplicamos a função de criamos em um groupby para que as agregações sejam feitas para cada cpf (obs.: esse passo está demorando cerca de 13 minutos para ser concluido)
clientes_engajados_jornada = clientes_engajados_jornada.groupby("num_cpf_hash").apply(agrupar_dados)

In [19]:
clientes_engajados_jornada

,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,cod_rating,ind_atrito,ind_engaj,restr
1,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,34734.06,5780.11,NaN,437.615385,2.0,NaN,13.0,NaN,10.0,0.0,NaN,1.0,3.0
4,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,4086.83,1400.27,NaN,749.076923,2.0,NaN,32.0,NaN,NaN,0.0,NaN,1.0,NaN
8,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,78661.03,5220.94,NaN,526.583333,1.0,NaN,14.0,NaN,NaN,0.0,NaN,1.0,NaN
9,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,116531.01,6663.52,NaN,359.307692,2.0,NaN,23.0,NaN,NaN,0.0,NaN,1.0,1.0
16,fffe7c6f3755240f56780edde1f91b70de574bf4e6e767...,9660.02,1752.27,NaN,553.153846,1.0,NaN,8.0,NaN,NaN,0.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12032483,000086b8a0b4c5f475b552eb75fc59b1720f33d7d67ec0...,793.93,689.36,NaN,652.000000,3.0,NaN,8.0,NaN,NaN,0.0,NaN,1.0,NaN
12032484,00008503efab22c4b17633da58b19557d135887e01db0e...,17926.23,5149.82,NaN,829.333333,3.0,NaN,38.0,NaN,1.0,0.0,NaN,1.0,1.0
12032485,00008166c6d8213d5372a4a6336322f14e7f15b982cf5d...,39931.26,2723.95,NaN,378.769231,3.0,NaN,11.0,NaN,8.0,0.0,NaN,1.0,8.0
12032487,0000567d0d7f3813befdc85b65838d685ec6d89bb067bd...,34645.77,4017.00,NaN,477.076923,1.0,NaN,15.0,NaN,4.0,7.0,NaN,1.0,4.0


In [20]:
#Trocamos todos os falores Na por zero
clientes_engajados_jornada = clientes_engajados_jornada.fillna(0)

In [21]:
clientes_engajados_jornada = clientes_engajados_jornada.drop_duplicates("num_cpf_hash")

In [23]:
# Importamos a biblioteca drive do google.colab
from google.colab import drive

# Estabelecemos os caminhos para o download poder ser executado
drive.mount(r'/content/drive')
path = r'/content/drive/My Drive/fe_agregacao_mediana_sprint_3.csv'

# Baixando nosso df no formato de csv em nosso drive
with open(path, 'w', encoding = 'utf-8-sig') as f:
  clientes_engajados_jornada.to_csv(f)

Mounted at /content/drive
